In [1]:
import pandas as pd
import ktrain

/home/j/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv('../data/annotated.csv')

In [3]:
data.head()

,Unnamed: 0,idx,msg_txt,annotation
0,0,0,"In Kyiv, fragments of a downed rocket damage...",1
1,1,1,"Kyiv region is attacked by drones again, air...",1
2,2,2,Explosions are also heard in the capital. Ai...,1
3,3,4,Kyiv region is attacked by drones - Kuleba ...,1
4,5,5,The rocket that fell in Shevchenkivskyi dist...,0


In [4]:
combat  = data[data['annotation']==1.0]

not_combat = data[data['annotation']==0.0]

In [5]:
data_sample = pd.concat([combat, not_combat])

In [6]:
data_sample.describe()

,Unnamed: 0,idx,annotation
count,953.000000,953.000000,953.000000
mean,724.318993,500.455404,0.295908
std,819.597210,294.900703,0.456689
min,0.000000,0.000000,0.000000
25%,246.000000,244.000000,0.000000
50%,501.000000,490.000000,0.000000
75%,766.000000,754.000000,1.000000
max,3000.000000,1027.000000,1.000000


## Modeling

Next, I'll load in a pre-trained transformer model. For now, I will use 'distilbert'.

In [6]:
train, val, preprocess = ktrain.text.texts_from_df(
    data_sample,
    text_column = 'msg_txt',
    label_columns = ['annotation'],
    val_df = None,
    max_features = 20000,
    maxlen = 128,
    val_pct = 0.1,
    ngram_range = 1,
    preprocess_mode = 'distilbert',
    verbose = 1
)

['not_annotation', 'annotation']
     not_annotation  annotation
333             1.0         0.0
599             0.0         1.0
524             0.0         1.0
587             0.0         1.0
674             1.0         0.0
['not_annotation', 'annotation']
     not_annotation  annotation
790             1.0         0.0
701             1.0         0.0
93              1.0         0.0
651             0.0         1.0
526             0.0         1.0
preprocessing train...
language: en
train sequence lengths:
	mean : 54
	95percentile : 170
	99percentile : 248


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 48
	95percentile : 136
	99percentile : 178


In [7]:
model = preprocess.get_classifier()
learner = ktrain.get_learner(model, train_data=train, val_data=val, batch_size=32)

In [8]:
history=learner.autofit(lr=1e-4, epochs=10, early_stopping=True)



begin training using triangular learning rate policy with max lr of 0.0001...
Epoch 1/10
27/27 [==============================] - 20s 239ms/step - loss: 0.5003 - accuracy: 0.7818 - val_loss: 0.2951 - val_accuracy: 0.8750
Epoch 2/10
27/27 [==============================] - 4s 163ms/step - loss: 0.2635 - accuracy: 0.8996 - val_loss: 0.2893 - val_accuracy: 0.8854
Epoch 3/10
27/27 [==============================] - 4s 162ms/step - loss: 0.1477 - accuracy: 0.9510 - val_loss: 0.2762 - val_accuracy: 0.8750
Epoch 4/10
27/27 [==============================] - 4s 167ms/step - loss: 0.1284 - accuracy: 0.9557 - val_loss: 0.3136 - val_accuracy: 0.9062
Epoch 4: early stopping
Weights from best epoch have been loaded into model.


In [9]:
predictor = ktrain.get_predictor(learner.model, preproc=preprocess)

In [10]:
validation = learner.validate(val_data=val, print_report=True)

3/3 [==============================] - 2s 150ms/step
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        66
           1       0.80      0.80      0.80        30

    accuracy                           0.88        96
   macro avg       0.85      0.85      0.85        96
weighted avg       0.88      0.88      0.88        96



In [11]:
predictor.save('distilbert')